<a href="https://colab.research.google.com/github/MohanVarmaDhana/Dhanamo-AI/blob/main/Semantic%20search%20engine%20using%20Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>